# Getting Started with RAiDER

**Author**: Jeremy Maurer, David Bekaert, Simran Sangha, Yang Lei - Jet Propulsion Laboratory, California Institute of Technology

This notebook provides an overview of how to get started using the RAiDER package for estimating tropospheric RADAR delays, and other functionality included in the **raiderDelay.py** program. We give an example of how to download and process delays using ERA-5 and HRRR weather models for the Los Angeles region. 

In this notebook, we will demonstrate how to:
- Download and install the RAiDER package
- Run RAiDER to generate a grid of delays over the Los Angeles region
    
<div class="alert alert-warning">
The initial setup (<b>Prep A</b> section) should be run at the start of the notebook. The overview sections do not need to be run in order. 
</div>

<div class="alert alert-danger">
<b>Potential Errors:</b> 
    
- GDAL uses "HDF5" driver instead of "netCDF/Network Common Data Format." Verify GDAL version >= 3.
- RAiDER needs to be installed to run this notebook
</div>


<div class="alert alert-info">
    <b>Terminology:</b>
    
- *Acquisition*: A single image acquired by a satellite at a particular time
- *Interferogram*: An unwrapped image containing the surface displacement accumulated between two acquisitions.
- *Weather model*: A reanalysis weather product defining temperature, pressure, and humidity on a regular grid in some coordinate system (e.g., at regular lat/lon intervals in the standard EPSG:4326 reference frame).
- *Delay*: The apparent displacement in an interferogram that occurs solely due to changes in weather conditions between two acquisitions.
    </div>
    

## Getting Started Using RAiDER

### The raiderDelay.py program
**`raiderDelay.py`** is a command-line program that allows for easy downloading and processing of tropospheric weather delays for InSAR correction. 

In [1]:
# Running raiderDelay.py -h will print the help menu
!raiderDelay.py -h

usage: raiderDelay.py [-h] --date DATELIST [DATELIST ...] --time TIME
                      (--latlon LAT LONG | --bbox S N W E | --station_file QUERY_AREA)
                      [--lineofsight LOS | --statevectors SV] [--dem DEM]
                      [--heightlvs HEIGHTLVS [HEIGHTLVS ...]]
                      [--model {ERA5,ERA5T,ERAI,MERRA2,WRF,HRRR,GMAO,HDF5,HRES,NCMR}]
                      [--files FILES [FILES ...]] [--weatherFiles WMLOC]
                      [--zref ZREF] [--parallel PARALLEL]
                      [--outformat OUTFORMAT] [--out OUT] [--download_only]
                      [--verbose]

Calculate tropospheric delay from a weather model.
Usage examples:
raiderDelay.py --date 20200103 --time 23:00:00 -b 39 40 -79 -78 --model ERA5 --zref 15000 -v
raiderDelay.py --date 20200103 --time 23:00:00 -b 39 40 -79 -78 --model ERA5 --zref 15000 --heightlvs 0 100 200 -v
raiderDelay.py --date 20200103 --time 23:00:00 --latlon test/scenario_1/geom/lat.dat test/scenario_1/geo

In [ ]:
# Get started quickly - Run an example command!
# This command will download the weather model ERA5 (change it to GMAO or HRRR if you do not have access to ERA-5)
# The download_only option specifies that the data will be downloaded from the remote server, but no other processing will occur
# The -b option gives a bounding box in SNWE format
# date and time of the delay to download is specified
# -v means "verbose," so RAiDER will print out extra information to the console
raiderDelay.py --date 20200103 --time 00:00:00 -b 39 40 -79 -78 --model ERA5 --download_only -v --out <output_directory>

#### Details of the command-line input options
RAiDER is quite flexible in terms of the input arguments it allows.  

### 1. Date or date list (--date)

This argument is required unless the weather model files are directly specified. The date(s) passed can be either: 
1) a single date, specified in psuedo-ISO 8601 format: 20180101, 2018-01-01, etc.  
2) a space-deliminated pair of dates, in which case all the dates between the pair listed will be downloaded and processed. E.g., '20180101 20190101'  
3) a space-delimited pair of dates plus an interval in days, to specify the date interval to download. E.g., '20180101 20190101 12'

### 2. Time of day (--time)

This argument is also required unless files are explicitly passed. Specify the time of day to download and process in psuedo-ISO 8601 format: 
1) T020000  
2) T02:00:00.000  
3) T0200  
4) 12:00:00  
5) 120000  
etc. 

### 3. Area of Interest (AOI) arguments

RAiDER accepts three different options as inputs for the AOI: 1) a bounding box, in which case the weather model grid nodes are used for x/y locations, 2) a station file, which should be a .csv containing at minimum the columns "Lat" and "Lon," and 3) ISCE-style latitude and longitude grid files in radar coordinates. Only one option should be used, and the arguments can be passed as described below.  

```--latlon```: <latitude file> <longitude file> in gridded radar coordinates. Example: ```--latlon lat.rdr lon.rdr```  
```--bbox```: A bounding box of numbers in South North West East format. Example: ```--bbox 41 42 -78 -77```  
```--station_file```: A comma-delimited file with at least the columns "Lat" and "Lon". Example: ```--station_file stations.csv```

### 4. Direction Information

This option can be used to specify a line-of-sight file such as those generated as outputs from the ISCE software package for InSAR (github.com/isce-framework/isce2). This should be a two-band GDAL-readable raster file containing the look vector components, with the incidence angle in band 1 and the heading in band 2.

This should be an ISCE-derived XML file or a shelve file containing state vectors specifying the orbit of the sensor. 

### 5. Height information

The DEM over the area of interest can be specified explicitly, or it will be downloaded on-the-fly. RAiDER will check the default location for a DEM (./geom), so if you download the DEM once it will not be downloaded again. 

### Height levels (--heightlvs)

This option specifies a list of heights, for which the delay will be calculated. The result is a full 3-D cube of delays at the specified lat/lon grid and height levels. 

### 6. Runtime parameters

### Weather model file directory location (-w/--weatherModelFileLocation)

Specifies a directory to store the original weather model files. If not specified, the default location is ./weather_files.

### Reference integration height (-z/--zref)

This option allows the user to specify the integration height when computing the total delay. The default is 15 km. 

### Output file format (--outformat)

This option is only used if the inputs are rasters or a bounding box, otherwise the output format is fixed (.csv file for station list, HDF5 file for height levels specified). Must be GDAL-compatible. 

### Output file directory (--out)

This specifies the location of the output files. If not specified the default is ./results

### Parallel Computation flag (-p, --no_parallel)

If specified, do not run in parallel. Off by default. 

### Download the weather model only (--download_only)

If specified, will only download the weather model and do nothing else. 

### Run in verbose mode (-v/--verbose)

Runs the code in verbose mode. Will save the weather model to a pickle file for inspection and create debugging plots.

## Examples using RAiDER
Below are some data demonstrations by using the debugging module of the RAiDER software. Here, the study area at Los Angles (latitude 34 deg, longitude -118 deg) is selected with a square box of 10 deg. See the optical image below.

The debugging plots have been generated for the weather models (ERA-5, ERA-5T, ERA-I, MERRA-2, GMAO, HRRR, ECMWF HRES, NCMR) on July 1st, 2018 at 00:00:00. For each weather model, the first debugging plot (with option 'pqt') shows the pressure/humidity/temperature at an altitude of 500 m (top row) and 15,000 m (middle row), and then the bottom row shows the vertical variation of pressure/humidity/temperature located specifically at the black point (Los Angeles).

The second debugging plot (with option 'wh') shows the wet and hydrostatic refractivity at the altitude of 500 m (top row) and 15,000 m (bottomw row).

![Optical Image at Los Angeles]()
<img src="img/optical.png" width="300" height="150">
\
\
![ERA-5 Weather Model Data 20180701T00:00:00]()
![ERA-5_weather_hgt500_and_15000m.pdf](img/ERA-5_weather_hgt500_and_15000m.png)
![ERA-5 Wet and Hydrostatic Refractivity 20180701T00:00:00]()
![ERA-5_refractivity_hgt500_and_15000m](img/ERA-5_refractivity_hgt500_and_15000m.png)
\
![ERA-5T Weather Model Data 20180701T00:00:00]()
![ERA-5T_weather_hgt500_and_15000m.pdf](img/ERA-5T_weather_hgt500_and_15000m.png)
![ERA-5T Wet and Hydrostatic Refractivity 20180701T00:00:00]()
![ERA-5T_refractivity_hgt500_and_15000m](img/ERA-5T_refractivity_hgt500_and_15000m.png)
\
![ERA-I Weather Model Data 20180701T00:00:00]()
![ERA-I_weather_hgt500_and_15000m.pdf](img/ERA-I_weather_hgt500_and_15000m.png)
![ERA-I Wet and Hydrostatic Refractivity 20180701T00:00:00]()
![ERA-I_refractivity_hgt500_and_15000m](img/ERA-I_refractivity_hgt500_and_15000m.png)
\
![MERRA-2 Weather Model Data 20180701T00:00:00]()
![MERRA2_weather_hgt500_and_15000m.pdf](img/MERRA2_weather_hgt500_and_15000m.png)
![MERRA-2 Wet and Hydrostatic Refractivity 20180701T00:00:00]()
![MERRA2_refractivity_hgt500_and_15000m](img/MERRA2_refractivity_hgt500_and_15000m.png)
\
![GMAO Weather Model Data 20180701T00:00:00]()
![GMAO_weather_hgt500_and_15000m.pdf](img/GMAO_weather_hgt500_and_15000m.png)
![GMAO Wet and Hydrostatic Refractivity 20180701T00:00:00]()
![GMAO_refractivity_hgt500_and_15000m](img/GMAO_refractivity_hgt500_and_15000m.png)
\
![HRRR Weather Model Data 20180701T00:00:00]()
![HRRR_weather_hgt500_and_15000m.pdf](img/HRRR_weather_hgt500_and_15000m.png)
![HRRR Wet and Hydrostatic Refractivity 20180701T00:00:00]()
![HRRR_refractivity_hgt500_and_15000m](img/HRRR_refractivity_hgt500_and_15000m.png)
\
![ECMWF HRES Weather Model Data 20180701T00:00:00]()
![HRES_weather_hgt500_and_15000m.pdf](img/HRES_weather_hgt500_and_15000m.png)
![ECMWF HRES Wet and Hydrostatic Refractivity 20180701T00:00:00]()
![HRES_refractivity_hgt500_and_15000m](img/HRES_refractivity_hgt500_and_15000m.png)
\
![NCMR Weather Model Data 20180701T00:00:00]()
![NCMR_weather_hgt500_and_15000m.pdf](img/NCMR_weather_hgt500_and_15000m.png)
![NCMR Wet and Hydrostatic Refractivity 20180701T00:00:00]()
![NCMR_refractivity_hgt500_and_15000m](img/NCMR_refractivity_hgt500_and_15000m.png)